# Explore daily pairs going up and down

- Load a preprocessed candle data containing all Uni v3 pairs on Polygon
- Explore daily negative and positive price movements, with volume filter

In [38]:
import pandas as pd

from tradingstrategy.client import Client
from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket

client = Client.create_jupyter_client()

chain_id = ChainId.polygon
time_bucket = TimeBucket.d1
exchange_slug = "uniswap-v3"

uni = exchanges.get_by_chain_and_slug(ChainId.polygon, "uniswap-v3")

# Filter out pair ids that belong to our target dataset
pairs_df = client.fetch_pair_universe().to_pandas()
pairs_df = pairs_df.loc[pairs_df["exchange_id"] == uni.exchange_id]

fpath = f"/tmp/{chain_id.get_slug()}-{exchange_slug}-candles-{time_bucket.value}.parquet"
candles_df = pd.read_parquet(fpath)

#candles=GroupedCandleUniverse(
#    candles_df,
#    index_automatically=False, # Preprocesed Parquet file   
#    fix_wick_threshold=False, # Preprocesed Parquet file   
#)

#data_universe = Universe(
#    exchanges=exchanges,
#    pairs=PandasPairUniverse(pairs_df),
#    candles=GroupedCandleUniverse(candles_df),
#)

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


### Interactive viewer mode

Set Plotly chart output mode to interactive viewing.

In [39]:
from tradeexecutor.utils.notebook import OutputMode, setup_charting_and_output

setup_charting_and_output(OutputMode.interactive, image_format="svg", max_rows=35)
#setup_charting_and_output(
#    OutputMode.static, 
#    image_format="png",
#    width=1500,
#    height=1200,    
#)

## Explore pairs



In [40]:
print(f"We have total {len(pairs_df)} pairs")

fee_groupby = pairs_df.groupby("fee")
for fee_tier, df in fee_groupby:
    print(f"Fee {fee_tier} BPS has {len(df)} pairs")

We have total 878 pairs
Fee 1 BPS has 75 pairs
Fee 5 BPS has 100 pairs
Fee 30 BPS has 343 pairs
Fee 100 BPS has 360 pairs


## Remove untradeable pairs


In [42]:
from tradingstrategy.pair import filter_for_stablecoins, StablecoinFilteringMode

# Remove pairs with expensive 1% fee tier
# Remove stable-stable pairs
tradeable_pairs_df = pairs_df.loc[pairs_df["fee"] <= 30]
tradeable_pairs_df = filter_for_stablecoins(tradeable_pairs_df, StablecoinFilteringMode.only_volatile_pairs)

# Narrow down candle data to pairs that are left after filtering
candles_df = candles_df.loc[candles_df["pair_id"].isin(tradeable_pairs_df["pair_id"])]

print(f"We have {len(tradeable_pairs_df)} tradeable pairs")

We have 484 tradeable pairs


## Daily up and down analysis

- Figure out number of pairs under a threshold that go up and down daily



In [78]:
import plotly.graph_objects as go
import plotly.express as px

volume_threshold = 10_000
momentun_threshold = 0.005

# Get all pair candles matching volume criteria
valid = candles_df.loc[candles_df["volume"] >= volume_threshold].sort_index() 

# Filter for momentum threshold to get rid of noise 
up = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] >= momentun_threshold
]
down = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] <= -momentun_threshold
]

up_df = pd.DataFrame({"count": up.groupby("timestamp").size(), "direction": "up"})
down_df = pd.DataFrame({"count": down.groupby("timestamp").size(), "direction": "down"})

df = pd.concat([up_df, down_df]).sort_index()
display(df)

fig = px.area(df, y="count", color="direction", title=f"Daily up/down pairs, {exchange_slug} on {chain_id.get_name()}, volume threshold {volume_threshold:,} USD, momentum threshold {momentun_threshold * 100:.02f}%")
fig.show()

,count,direction
timestamp,,
2022-07-07,7,up
2022-07-08,4,up
2022-07-08,4,down
2022-07-09,4,down
2022-07-10,7,down
...,...,...
2023-11-06,11,down
2023-11-07,16,up
2023-11-07,15,down


# Weekly up and down

- Same as above, but aggregated weekly

In [ ]:
# Get all pair candles matching volume criteria
from tradingstrategy.utils.groupeduniverse import resample_candles

weekly_candles = resample_candles(candles_df, pd.Timedelta(days=7))
valid = candles_df.loc[candles_df["volume"] >= volume_threshold * 7].sort_index() 

# Filter for momentum threshold to get rid of noise 
up = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] >= momentun_threshold
]
down = valid.loc[
    (valid["close"] - valid["open"]) / valid["open"] <= -momentun_threshold
]

up_df = pd.DataFrame({"count": up.groupby("timestamp").size(), "direction": "up"})
down_df = pd.DataFrame({"count": down.groupby("timestamp").size(), "direction": "down"})

df = pd.concat([up_df, down_df]).sort_index()
display(df)

fig = px.area(df, y="count", color="direction", title=f"Weekly up/down pairs, {exchange_slug} on {chain_id.get_name()}, volume threshold {volume_threshold:,} USD, momentum threshold {momentun_threshold * 100:.02f}%")
fig.show()